# Ward level census data

- [Method of travel to  work (2011)](https://www.nomisweb.co.uk/census/2011/qs701ew)
- [Car and van ownership (2011)](https://www.nomisweb.co.uk/census/2011/qs416ew)
- [Deprivation dimensions (2011)](https://www.nomisweb.co.uk/census/2011/qs119ew)
- [Ward boundaries (2011)](https://statistics.ukdataservice.ac.uk/dataset/2011-census-geography-boundaries-wards-and-electoral-divisions)
- [Ward changes 2011 - 2021](https://geoportal.statistics.gov.uk/datasets/ons::lsoa-2011-to-lsoa-2021-to-local-authority-district-2022-lookup-for-england-and-wales/explore)
- [Oxford ward population (2011)](https://www.oxford.gov.uk/downloads/file/1082/ward_population)
  

In [111]:
import pathlib
import kml2geojson
import json
import geopandas as gpd
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

%matplotlib inline


In [2]:
car_ownership_data_file = 'data/car_ownership_2011.csv'
travel_to_work_data_file = 'data/travel_to_work_2011.csv'

df_car_ownership = pd.read_csv(car_ownership_data_file)
df_travel_to_work = pd.read_csv(travel_to_work_data_file)

In [3]:
df_car_ownership.head()

,date,geography,geography code,Rural Urban,Method of Travel to Work: All categories: Method of travel to work; measures: Value,Method of Travel to Work: Work mainly at or from home; measures: Value,"Method of Travel to Work: Underground, metro, light rail, tram; measures: Value",Method of Travel to Work: Train; measures: Value,"Method of Travel to Work: Bus, minibus or coach; measures: Value",Method of Travel to Work: Taxi; measures: Value,"Method of Travel to Work: Motorcycle, scooter or moped; measures: Value",Method of Travel to Work: Driving a car or van; measures: Value,Method of Travel to Work: Passenger in a car or van; measures: Value,Method of Travel to Work: Bicycle; measures: Value,Method of Travel to Work: On foot; measures: Value,Method of Travel to Work: Other method of travel to work; measures: Value,Method of Travel to Work: Not in employment; measures: Value
0,2011,Bank Top,E05001553,Total,2953,75,0,50,150,23,2,882,161,46,408,16,1140
1,2011,Central,E05001554,Total,3123,63,0,49,185,14,3,803,130,51,484,8,1333
2,2011,Cockerton East,E05001555,Total,3930,79,1,31,193,26,9,1698,184,67,354,13,1275
3,2011,Cockerton West,E05001556,Total,2785,35,0,21,233,34,8,804,106,44,186,8,1306
4,2011,College,E05001557,Total,2970,141,4,74,52,7,6,1304,92,48,362,13,867


In [55]:
## Need to get list of geography codes for Oxford
ward_data_file = 'data/infuse_ward_lyr_2011.csv'
df_ward_data = pd.read_csv(ward_data_file)

In [72]:
# Geo codes for Oxford wards are sequential from E05006545 (Barton and Sandhills) to E05006568 (Wolvercote)
# There are 22 wards, which does contradict the OCC spreadsheet, but I'm going to assume that the OCC spreadsheet is wrong
# and that the infuse data is correct
# E05006566 and E05006554 are missing from the sequence for some reason 
valid_geo_codes = set([f'E050065{str(i).zfill(2)}' for i in range(45, 69)])
df_ward_data = df_ward_data[df_ward_data['geo_code'].isin(valid_geo_codes)]

In [73]:
assert len(df_ward_data) == 22

In [93]:
# Let's print a nice map of these wards 
# We'll use the kml2geojson library to convert the kml file to geojson
# Then we'll use the geojson library to print the map
ward_data_kml = 'data/infuse_ward_lyr_2011.kml' 
ward_data_geojson_file = 'data/oxford_ward_data.geojson'

if not pathlib.Path(ward_data_geojson_file).exists():
    ward_data_geojson = kml2geojson.main.convert(ward_data_kml)  # Takes 90s to run, only needs to be run once to generate the geojson file
    # Filter geojson to only include the wards we want
    ward_data_geojson[0]['features'] = [feature for feature in ward_data_geojson[0]['features'] if feature['properties']['geo_code'] in valid_geo_codes]
    # Dump this geojson to a file
    with open(ward_data_geojson_file, 'w') as f:
        json.dump(ward_data_geojson, f)
else:
    with open(ward_data_geojson_file, 'r') as f:
        ward_data_geojson = json.load(f)

In [94]:
ward_data_geojson[0]['features'][0]['properties']

{'name': 'Headington Hill and Northway',
 'fill-opacity': 0.0,
 'stroke': '#ff0000',
 'stroke-opacity': 1.0,
 'geo_code': 'E05006552',
 'geo_label': 'Headington Hill and Northway',
 'label': 'E92000001E05006552'}

In [112]:
df_ward_geo = gpd.read_file(ward_data_geojson_file)

DriverError: 'data/oxford_ward_data.geojson' not recognized as a supported file format.

In [110]:
# Print map of Oxford wards from geojson
ward_data_geojson[0]['features'][0]['geometry']

{'type': 'Polygon',
 'coordinates': [[[-1.229816219403281, 51.77006743075064],
   [-1.229814723611891, 51.77006846393775],
   [-1.229599710898753, 51.77021693947559],
   [-1.229567921441465, 51.77024890185933],
   [-1.229562646039539, 51.770254873444024],
   [-1.229546686960516, 51.77027293118597],
   [-1.229512408704708, 51.770307214970536],
   [-1.229503828341077, 51.77031578134872],
   [-1.229497164001666, 51.77032076378548],
   [-1.229450176012245, 51.770355890374624],
   [-1.22941897427485, 51.77037885634121],
   [-1.22939102950098, 51.77039778859443],
   [-1.229339807541553, 51.77043133191078],
   [-1.229331805423773, 51.77043657530834],
   [-1.229323700797232, 51.770441880972264],
   [-1.22926866433924, 51.77048240343252],
   [-1.229245971930371, 51.77051203368084],
   [-1.229225672937366, 51.770533875218796],
   [-1.229221052198215, 51.770538844060454],
   [-1.229188942089399, 51.770563224627445],
   [-1.229143734394665, 51.77058483994374],
   [-1.229108025640956, 51.7706086304